In [ ]:
import polars as pl

In [ ]:
restaurantes = pl.read_csv('/content/drive/MyDrive/Alura/restaurantes.csv', encoding='utf8')

In [ ]:
restaurantes.head()

In [ ]:
restaurantes.glimpse()

### Convertendo valores numéricos

In [ ]:
restaurantes.with_columns(
    pl.col('approx_cost(for two people)')
    .str.replace(',', '')
)

In [ ]:
restaurantes.with_columns(
    pl.col('approx_cost(for two people)')
    .str.replace(',', '')
    .cast(pl.Float64)
)

In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('approx_cost(for two people)')
    .str.replace(',', '')
    .cast(pl.Float64)
)

### Coletando estatísticas

In [ ]:
restaurantes.select(
    pl.col('approx_cost(for two people)').max().alias('max'),
    pl.col('approx_cost(for two people)').min().alias('min'),
    pl.col('approx_cost(for two people)').mean().round(2).alias('média')
)

## Usando o Regex

In [ ]:
restaurantes.with_columns(
    pl.col('rate')
    .str.extract(r'^([\d\.]+)')
)

In [ ]:
restaurantes.with_columns(
    pl.col('rate')
    .str.extract(r'^([\d\.]+)')
    .cast(pl.Float64)
)


In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('rate')
    .str.extract(r'^([\d\.]+)')
    .cast(pl.Float64)
)

In [ ]:
restaurantes.select(
    pl.col('rate').max().alias('max'),
    pl.col('rate').min().alias('min')
)

Os valores obtidos revelam a amplitude das avaliações no mercado, desde os restaurantes mais bem avaliados até aqueles com pior reputação. Essa informação é crucial para entendermos o nível de exigência dos clientes e identificamos benchmarks de qualidade. Com essa base, podemos aprofundar a análise investigando a relação entre avaliações e outros fatores, como faixa de preço, tipo de restaurante e pratos mais populares, garantindo insights valiosos para o posicionamento da nova franquia.

## Convertendo textos para listas

Compreender as preferências gastronômicas dos clientes é essencial para definir o cardápio da nova franquia. Restaurantes bem-sucedidos geralmente possuem pratos que se destacam e atraem clientes recorrentes.<br>
Para garantir uma análise precisa, precisamos limpar e estruturar os dados das colunas `"dish_liked"` (pratos favoritos) e `"cuisines"` (tipos de culinária). Isso envolve dividir as listas de itens e remover espaços desnecessários, tornando os dados mais organizados para futuras análises.

In [ ]:
restaurantes.with_columns(
    pl.col('dish_liked')
    .cast(pl.List(pl.String)),
    pl.col('cuisines')
    .cast(pl.List(pl.String))
)

In [ ]:
restaurantes.with_columns(
    pl.col('dish_liked')
    .str.split(','),
    pl.col('cuisines')
    .str.split(',')
)

In [ ]:
restaurantes.with_columns(
    pl.col('dish_liked')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars()),
    pl.col('cuisines')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars())
)

In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('dish_liked')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars()),
    pl.col('cuisines')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars())
)

## Identificando dados únicos

O cardápio de um restaurante é um dos principais fatores que influenciam a experiência do cliente e a decisão de comsumo. Analisar os itens disponíveis pode revelar padrões e tendências gastronômicas que ajudam a definir o melhor mix de produtos para a nova franquia.<br>
No entanto, os dados brutos podem conter formatações indesejadas, com cochetes e espaços extras. Para garantir um análise precisa, primeiro limpamos a estruturamos a coluna "menu_item", transformando os itens do cardápio em listas organizadas.

In [ ]:
restaurantes.select(
    pl.col('menu_item').unique()
)

In [ ]:
restaurantes.select(
    pl.col('menu_item').unique()
).item(0,0)

In [ ]:
restaurantes.with_columns(
    pl.col('menu_item')
    .str.replace('\[', '')
    .str.replace('\]', '')
)

In [ ]:
restaurantes.with_columns(
    pl.col('menu_item')
    .str.replace('\[', '')
    .str.replace('\]', '')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars())
)

In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('menu_item')
    .str.replace('\[', '')
    .str.replace('\]', '')
    .str.split(',')
    .list.eval(pl.element().str.strip_chars())
)

In [ ]:
restaurantes.filter(
    pl.col('menu_item').list.len() > 1
).select(
    pl.col('menu_item')
).item(0,0).to_list()



## Trabalhando com tipos de dados aninhados

A experiência do cliente é um dos fatores mais importantes para o sucesso de um restaurante. As avaliações e comentários fornecem insights valiosos sobre a percepção do público, ajudando a identificar pontos fortes e áreas de melhoria.<br>
Para garantir que os dados sejam analisados corretamente, primeiro precisamos limpar e estruturar a coluna `"reviews_list"`, removendo caracteres indesejados e separando os comentários individuais. Isso tornará a análise mais eficiente e permitirá extrais insights relevantes.

In [ ]:
restaurantes.select(
    pl.col('reviews_list')
).item(0,0)

In [ ]:
restaurantes.with_columns(
    pl.col('reviews_list')
    .str.replace('\[', '')
    .str.replace('\]', '')
    .str.replace('\(', '')
).select(
    pl.col('reviews_list')
).item(0,0)

In [ ]:
restaurantes.with_columns(
    pl.col('reviews_list')
    .str.replace('\[', '')
    .str.replace('\]', '')
    .str.replace('\(', '')
    .str.split('), (')
)

In [ ]:
restaurantes = restaurantes.with_columns(
    pl.col('reviews_list')
    .str.replace('\[', '')
    .str.replace('\]', '')
    .str.replace('\(', '')
    .str.split('), (')
)


## Conhecendo a proporção de categorias

No setor de restaurantes, a forma como os clientes fazem pedidos e reservas pode impactar diretamente a experiência do usuário e o volume de vendas. Com a popularização dos aplicativos de entrega e reserva online, entender a frequência de pedidos online e a reserva de mesas pode ajudar na definição do modelo de negócio da nova franquia.<br>
Para isso, realizamos um análise da proporção de restaurantes que oferecem pedidos online e reserva de mesas, verificando a distribuição dessas opções no mercado.

In [ ]:
restaurantes.select(
    [pl.col('online_order').value_counts()]
)

In [ ]:
restaurantes.select(
    [pl.col('online_order').value_counts()]
).unnest('online_order')

In [ ]:
total_registros = restaurantes.height
total_registros

In [ ]:
restaurantes.select([
    pl.col('online_order').value_counts()
]).unnest('online_order').with_columns(
    (pl.col('count')/total_registros *100).round(2)
)

In [ ]:
restaurantes.select([
    pl.col('book_table').value_counts()
]).unnest('book_table').with_columns(
    (pl.col('count')/total_registros *100).round(2)
)

## Descobrindo tipos de restaurantes mais comuns

O tipo de restaurante é um fator essencial para entender o posicionamento no mercado e as preferências do público. Alguns formatos, como casual dining, cafés ou fast food, podem ser mais populares e atrair mairo volume de clientes, equanto outros, como restaurantes fine dinning, pode ter um ticket médio mais elevado.<br>
Para identicar as categorias mais comuns, analisamos a distribuição dos tipos de restaurantes, destacando os 10 mais frequentes na base de dados.

In [ ]:
restaurantes.select([
    pl.col('rest_type').n_unique(),
    pl.col('listed_in(type)').n_unique()
])

In [ ]:
restaurantes.select([
    pl.col('listed_in(type)').value_counts()
]).unnest('listed_in(type)')

In [ ]:
restaurantes.select([
    pl.col('listed_in(type)').value_counts()
]).unnest('listed_in(type)').sort('count', descending=True)

In [ ]:
restaurantes.select([
    pl.col('rest_type').value_counts()
]).unnest('rest_type').sort('count', descending=True).head(10)

## Analisando a relação de variáveis

Para entender melhor o mercado e o comportamento dos clientes, analisamos a relação entre três variáveis-chave:
- Avaliação (rate) - indica a percepção dos clientres sobre a qualidade do restaurante.
- Quanditdade de votos (votes) - Mostra o nível de engajamento dos clientes.
- Custo apoximado para duas pessoas - Reflete o posicionamento de preço dos restaurantes.<br>

Primeiro, exploramos estatísticas descritivas dessas colunas para entender a distribuição dos dados. Em seguida, criamos gráficos de dispersão para identificar padrões e correlações entre esses fatores.

In [ ]:
restaurantes.select([
    pl.col('rate'),
    pl.col('votes'),
    pl.col('approx_cost(for two people)')
]).describe()

In [ ]:
!pip install 'vegafusion [embed]>=1.5.0'

In [ ]:
!pip install 'vl-convert-python>=1.6.0'

In [ ]:
import altair as alt
alt.data_transformers.enable('vegafusion')

In [ ]:
restaurantes.plot.scatter(x = 'rate', y = 'approx_cost(for two people)').properties(width=800, height = 350, title = 'Avaliação média x custo aproximado para duas pessoas')

In [ ]:
restaurantes.plot.scatter(x = 'approx_cost(for two people)', y = 'votes').properties(width=800, height = 350, title = ' Custo aproximado para duas pessoas x Quantidade de avaliações')

In [ ]:
restaurantes.plot.scatter(x = 'rate', y = 'votes').properties(width=800, height = 350, title = 'Avaliação média x Quantidade de avaliações')

## Entendendo a distribuição com gráficos

Agora que já analisamos a relação entre as variáveis, seria interessante analisar o comportamento geral da distribuição de cada uma delas, buscando por padrões que possam destacar restrantes com qualidade maior que os demais.<br>
Vamos utilizar histogramas para avaliar a distribuição dos clientes, custo aproximado para duas pessoas e quantidade de avaliações.

In [ ]:
restaurantes.plot.bar(x = alt.X('rate', bin = alt.Bin(maxbins = 20)), y = alt.Y('count()')).properties(width = 800, height = 350, title = 'Distribuição de notas dadas pelos clientes')

In [ ]:
restaurantes.plot.bar(x = alt.X('approx_cost(for two people)', bin = alt.Bin(maxbins = 20)), y = alt.Y('count()')).properties(width = 800, height = 350, title = 'Distribuição de custo aproximado para duas pessoas')

In [ ]:
restaurantes.plot.bar(x = alt.X('votes', bin = alt.Bin(maxbins = 20)), y = alt.Y('count()')).properties(width = 800, height = 350, title = 'Distribuição de avaliações de clientes')

In [ ]:
restaurantes.filter(
    pl.col('votes') > 200
).plot.bar(x = alt.X('votes', bin = alt.Bin(maxbins = 20)), y = alt.Y('count()')).properties(width = 800, height = 350, title = 'Distribuição de avaliações de clientes (acima de 200 avaliações)')